In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder,StandardScaler,PowerTransformer, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, cross_val_predict, train_test_split,GridSearchCV,cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR


from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.feature_selection import SelectKBest,SelectPercentile,f_classif,f_regression,mutual_info_regression,mutual_info_classif,SelectFromModel,RFE

from xgboost import XGBRegressor
from xgboost import plot_importance


import optuna

#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('max_columns',100)
pd.set_option('max_rows',900)

pd.set_option('max_colwidth',200)

df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

In [4]:
df.shape

In [5]:
df.info()

In [7]:
df.nunique()

In [10]:
#numerical = df.drop(['stroke'], axis= 1).select_dtypes('number').columns
categorical = df.select_dtypes('object').columns

#print(f'Numerical_Columns: {df[numerical].columns}')
#print('\n')
print(f'Categorical_Columns: {df[categorical].columns}')

In [11]:
df.smoking_status.value_counts()

In [13]:
df.work_type.value_counts()

In [17]:
sns.heatmap(df.corr(), annot=True)

In [26]:
df=df.drop('id', axis=1)


In [25]:
df.columns

In [29]:
numerical = df.drop(['stroke'], axis= 1).select_dtypes('number').columns
numerical

In [30]:
index = 0
plt.figure(figsize=(20,20))
for feature in df.select_dtypes('number').columns:
    plt.figure()
    plt.boxplot(df[feature],vert=False)
    plt.ylabel(feature)
    plt.show()

In [31]:
df1 = pd.get_dummies(df, drop_first=True)
df1.head()

In [34]:
df.shape

In [35]:
df.isnull().sum()

In [38]:
df.stroke.value_counts()

In [39]:
df["bmi"].fillna(df.groupby(["age",'gender']).bmi.transform("mean"), inplace=True)

In [41]:
df.isnull().sum()

In [48]:
df=df.dropna()

In [49]:
df.isnull().sum()

In [51]:
df.avg_glucose_level.sort_values()


In [ ]:
LogisticRegression

In [52]:
df.shape

In [54]:
df = pd.get_dummies(df, drop_first=True)
df.head()

In [55]:
df2=df.copy()

## LogisticRegression

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve, roc_auc_score, auc, roc_curve

In [94]:
X= df.drop('stroke', axis=1)
y= df['stroke']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [95]:
scaler = MinMaxScaler()

In [96]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [97]:
log_model = LogisticRegression(class_weight = "balanced")

In [98]:
log_model.fit(X_train_scaled,y_train)
y_pred = log_model.predict(X_test_scaled)
y_pred

In [65]:
#Test_Size = 0.3 ile
#print(confusion_matrix(y_test, y_pred))
#print(classification_report(y_test, y_pred))

In [99]:
#TEST_SIZE =0.2 ILE

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [100]:
y_pred_proba = log_model.predict_proba(X_test_scaled)

In [101]:
fp_rate, tp_rate, thresholds = roc_curve(y_test, y_pred_proba[:,1])

In [102]:
optimal_idx = np.argmax(tp_rate - fp_rate)
optimal_threshold = thresholds[optimal_idx]
optimal_threshold

In [103]:
test_data = pd.concat([X_test, y_test], axis=1)

In [104]:
test_data["pred_proba"] = y_pred_proba[:,1]

In [105]:
test_data["pred"] = y_pred

In [106]:
test_data["pred2"] = test_data["pred_proba"].apply(lambda x : 1 if x >= optimal_threshold else 0)

In [107]:
y_pred2 = test_data["pred2"]

In [79]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [108]:
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

## KNN

In [90]:
from sklearn.neighbors import KNeighborsClassifier

In [91]:
from sklearn.model_selection import GridSearchCV

In [119]:
knn_grid = KNeighborsClassifier()

In [120]:
k_values= range(1,30)

In [121]:
param_grid = {"n_neighbors":k_values, "p": [1,2], "weights": ['uniform', "distance"]}

In [122]:
knn_grid_model = GridSearchCV(knn_grid, param_grid, cv=10, scoring= 'accuracy')

In [124]:
knn_grid_model.fit(X_train_scaled, y_train)

In [127]:
knn_grid_model.best_params_

In [128]:
knn = KNeighborsClassifier(n_neighbors=7, p =2)

knn.fit(X_train_scaled,y_train)
pred = knn.predict(X_test_scaled)

print('WITH K=21')
print('\n')
print(confusion_matrix(y_test, pred))
print('\n')
print(classification_report(y_test, pred))

In [125]:
test_error_rates = []


for k in range(1,30):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train_scaled,y_train) 
   
    y_pred_test = knn_model.predict(X_test_scaled)
    
    test_error = 1 - accuracy_score(y_test,y_pred_test)
    test_error_rates.append(test_error)

In [126]:
plt.figure(figsize=(10,6),dpi=200)
plt.plot(range(1,30),test_error_rates,label='Test Error')
plt.legend()
plt.ylabel('Error Rate')
plt.xlabel("K Value")

## ROC

In [129]:
from sklearn.metrics import precision_recall_curve, plot_precision_recall_curve, plot_roc_curve, roc_auc_score, roc_curve

In [135]:
knn_model = KNeighborsClassifier(n_neighbors=7).fit(X_train_scaled, y_train)

In [136]:
accuracy_score(y_test,y_pred)

In [137]:
confusion_matrix(y_test,y_pred)

In [139]:
print(classification_report(y_test,y_pred))